In [2]:
import os
import csv
import glob
import random
import string
import numpy as np
import pandas as pd
from io import StringIO

# -------------------------------------------------
# CONFIG
# -------------------------------------------------
folder_path = r"D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV"
output_folder = os.path.join(folder_path, "Ultrastitch")
os.makedirs(output_folder, exist_ok=True)

PROBE_PITCH_MM = 50  # 🔴 Probe spacing

# -------------------------------------------------
# SMALL RANDOM ID MAKER
# -------------------------------------------------
def make_small_id():
    return "S#" + ''.join(random.choices(string.ascii_uppercase, k=5))

all_properties = []

# -------------------------------------------------
# LOAD ANY SCAN AS CSV TEXT
# -------------------------------------------------
def load_scan_as_csv_text(filepath):
    ext = os.path.splitext(filepath)[1].lower()
    metadata = {}

    # -----------------------
    # TXT
    # -----------------------
    if ext == ".txt":
        matrix_lines = []

        with open(filepath, "r") as f:
            for line in f:
                line = line.strip()

                if line.startswith("mm"):
                    matrix_lines.append(line)
                    break

                if "=" in line:
                    k, v = [x.strip() for x in line.split("=", 1)]
                    metadata[k] = v

            for line in f:
                matrix_lines.append(line.strip())

        return "\n".join(matrix_lines), metadata

    # -----------------------
    # CSV
    # -----------------------
    if ext == ".csv":
        with open(filepath, "r") as f:
            return f.read(), metadata

    # -----------------------
    # EXCEL (Data sheet)
    # -----------------------
    if ext in [".xls", ".xlsx"]:
        df = pd.read_excel(filepath, sheet_name="Data", header=0)

        # Convert headers: P-1 → 0, P-2 → 50 ...
        new_cols = []
        for i, col in enumerate(df.columns):
            if i == 0:
                new_cols.append("mm")
            else:
                new_cols.append((i - 1) * PROBE_PITCH_MM)

        df.columns = new_cols

        buffer = StringIO()
        df.to_csv(buffer, index=False)
        return buffer.getvalue(), metadata

    raise ValueError(f"Unsupported file type: {filepath}")

# -------------------------------------------------
# PROCESS SCAN FILES
# -------------------------------------------------
scan_files = glob.glob(os.path.join(folder_path, "*.*"))

for scan_file in scan_files:
    if not scan_file.lower().endswith((".txt", ".csv", ".xls", ".xlsx")):
        continue

    print(f"Processing scan file: {scan_file}")

    properties = {
        "id": make_small_id(),
        "scan_details": os.path.basename(scan_file),
        "nominal_thk": None,
        "min_thk": None,
        "max_thk": None,
        "x": 0,
        "y": 0,
        "width": None,
        "height": None,
        "rotation": 0
    }

    # -----------------------
    # LOAD FILE AS CSV TEXT
    # -----------------------
    try:
        matrix_text, metadata = load_scan_as_csv_text(scan_file)
    except Exception as e:
        print(f"❌ Failed to load {scan_file}: {e}")
        continue

    # -----------------------
    # NUMPY PARSE
    # -----------------------
    data = np.genfromtxt(
        StringIO(matrix_text),
        delimiter=",",
        dtype=float,
        filling_values=np.nan
    )

    if data.ndim != 2 or data.shape[0] < 2 or data.shape[1] < 2:
        print(f"❌ Invalid matrix in file: {scan_file}")
        continue

    # -----------------------
    # AXES + MATRIX
    # -----------------------
    raw_x = data[0, 1:]

    # If headers were P-1, P-2 etc → NaN
    if np.isnan(raw_x).all():
        x_values = np.arange(data.shape[1] - 1) * PROBE_PITCH_MM
    else:
        x_values = raw_x

    y_values = data[1:, 0]
    t_values = data[1:, 1:]

    # -----------------------
    # REVERSE Y AXIS
    # -----------------------
    y_values = y_values[::-1]
    t_values = t_values[::-1, :]

    # -----------------------
    # GEOMETRY
    # -----------------------
    properties["width"] = float(np.nanmax(x_values) - np.nanmin(x_values))
    properties["height"] = float(np.nanmax(y_values) - np.nanmin(y_values))

    # -----------------------
    # THICKNESS STATS
    # -----------------------
    properties["min_thk"] = float(np.nanmin(t_values))
    properties["max_thk"] = float(np.nanmax(t_values))
    properties["nominal_thk"] = float(metadata.get("Max Thickness (mm)", 0))

    # -----------------------
    # SAVE MATRIX CSV
    # -----------------------
    csv_filename = properties["id"] + ".csv"
    csv_filepath = os.path.join(output_folder, csv_filename)

    with open(csv_filepath, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)

        writer.writerow(["mm"] + list(x_values))

        for y, row in zip(y_values, t_values):
            writer.writerow([y] + row.tolist())

    all_properties.append(properties)

# -------------------------------------------------
# SAVE MASTER PROPERTIES CSV
# -------------------------------------------------
properties_csv = os.path.join(output_folder, "scan_properties.csv")

if all_properties:
    with open(properties_csv, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        header = list(all_properties[0].keys())
        writer.writerow(header)
        for p in all_properties:
            writer.writerow([p[h] for h in header])

    print(f"\n✅ Saved all properties → {properties_csv}")
else:
    print("\n⚠ No valid scan files found.")


Processing scan file: D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV\Scan No-100-(N-25)FE.csv
Processing scan file: D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV\Scan No-101-(N-25)FE.csv
Processing scan file: D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV\Scan No-102-(N-25)FE.csv
Processing scan file: D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV\Scan No-103-(N-25)FE.csv
Processing scan file: D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV\Scan No-104-(N-25)FE.csv
Processing scan file: D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV\Scan No-105-(N-25)FE.csv
Processing scan file: D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV\Scan No-106-(N-25)FE.csv
Processing scan file: D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV\Scan No-107-(N-25)FE.csv
Processing scan file: D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV\Scan No-108-(N-25)FE.csv
Processing scan file: D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV\Scan No-109-(N-25)FE.csv
Processing scan file: D:\Tpac Workdir\Ultratech Kiln\Tadipatri\CSV\Scan No-110-(